# Examples for how to use our ecological-RL API

Our approach seeks to minimize the amount of computational information that the user
needs to provide in order to get an RL algorithm up and running on their population
dynamics control problem.

## 1. Using ray RLLib to train

The class `ray_trainer_api.ray_trainer` may be used for defining, tuning, and training an agent using the ray RLLib framework.

In [ ]:
#! pip install ray[rllib]
#! pip install gymnasium
#! pip install numpy
#! pip install pandas
#! pip install scipy

from ray_trainer_api import ray_trainer
from dyn_fns import threeSp_1


### Ecological input

The cell below translates the ecological data defining the control problem to the format that our classes use.

The `metadata` dictionary encapsulates most of the information of the control problem, except for the actual dynamics of the system. `dyn_fn` encapsulates the dynamics of the system (note that the number of arguments of this function must match `metadata['n_sp']`).

In [ ]:
metadata = {
	#
	# structure of ctrl problem
	'name': 'threeSp_1',
	'n_sp':  3,
	'n_act': 2,
	'_harvested_sp': [0,1],
	#
	# about episodes
	'init_pop': np.float32([0.5, 0.5, 0.1]),
	'reset_sigma': 0.01,
	'tmax': 1000,
	#
	# about dynamics / control
    'extinct_thresh': 0.05,
    'penalty_fn': lambda t: - 1000 / t,
	'var_bound': 4,
	'_costs': np.zeros(2, dtype=np.float32),
	'_prices': np.ones(2, dtype=np.float32),
}

params = {
	'c': np.random.choice([0.2, 0.25, 0.3]),
	'D': np.random.choice([0.05, 0.1, 0.15]),
	'd_z': np.random.choice([0.2, 0.3, 0.4]),
	'K_x': np.random.choice([0.9, 1, 1.1]),
	'LV_xy': np.random.choice([0.05, 0.1, 0.15]),
	'r_x': np.random.choice([0.9, 1, 1.1]),
	'r_y': np.random.choice([0.9, 1, 1.1]),
	'r_z': np.random.choice([0.9, 1, 1.1]),
	#
	'sigma_x': 0.1,
	'sigma_y': 0.1,
	'sigma_z': 0.1,
}

def dyn_fn(x,y,z):
	global params
	return threeSp_1(x,y,z,_params)

### Training

With the previous setup, we may define our trainer and train it as shown below.

In [ ]:
RT = ray_trainer(
	algo_name="ppo", 
	config={
        'metadata': metadata,
        'dyn_fn': dyn_fn,
    },
)
agent = RT.train(iterations=2)